In [30]:
import plotly.express as px
from utz import *
from ctbk import *
from geopy.distance import distance

### Load August 2022 data

In [2]:
month = '202208'
path = f's3/ctbk/normalized/{month}.parquet'
if not exists(path):
    path = f's3://ctbk/normalized/{month}.parquet'
d = read_parquet(path)
d

,Ride ID,Rideable Type,Start Time,Stop Time,Start Station Name,Start Station ID,End Station Name,End Station ID,Start Station Latitude,Start Station Longitude,End Station Latitude,End Station Longitude,Gender,User Type,Start Region,End Region
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,0,Subscriber,JC,JC
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,0,Subscriber,HB,HB
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,0,Subscriber,HB,HB
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,0,Subscriber,JC,JC
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,0,Subscriber,HB,JC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681304,231398EE256269B0,classic_bike,2022-08-17 17:48:50,2022-08-17 18:06:13,27 St & Hunter St,6310.06,E 54 St & 1 Ave,6608.09,40.748500,-73.941275,40.756265,-73.964179,0,Subscriber,NYC,NYC
3681305,2F9EC3A7F60523B1,classic_bike,2022-08-08 23:02:58,2022-08-08 23:19:10,W 22 St & 10 Ave,6306.06,W 15 St & 6 Ave,5989.02,40.746920,-74.004519,40.738046,-73.996430,0,Subscriber,NYC,NYC
3681306,094B17D42252E33E,classic_bike,2022-08-27 11:10:59,2022-08-27 11:14:03,27 St & Hunter St,6310.06,11 St & 43 Ave,6438.04,40.748500,-73.941275,40.751907,-73.947912,0,Subscriber,NYC,NYC
3681307,9CEB6714CBEC7386,classic_bike,2022-08-29 22:55:27,2022-08-29 23:13:10,Washington Pl & Broadway,5755.01,Canal St & Rutgers St,5303.08,40.729039,-73.994046,40.714275,-73.989900,0,Subscriber,NYC,NYC


### Parse Station Names, IDs, Lat/Lngs

In [3]:
columns = {
    'Start Station ID': 'Station ID', 
    'Start Station Name': 'Station Name',
    'Start Station Latitude': 'Latitude',
    'Start Station Longitude': 'Longitude',
}
starts = (
    d[columns.keys()]
    .rename(columns=columns)
)
starts['Start'] = True
columns ={
    'End Station ID': 'Station ID', 
    'End Station Name': 'Station Name',
    'End Station Latitude': 'Latitude',
    'End Station Longitude': 'Longitude',
}
ends = (
    d[columns.keys()]
    .rename(columns=columns)
)
ends['Start'] = False

station_entries = pd.concat([starts, ends])
station_entries

,Station ID,Station Name,Latitude,Longitude,Start
0,JC014,Columbus Drive,40.718355,-74.038914,True
1,HB103,South Waterfront Walkway - Sinatra Dr & 1 St,40.736982,-74.027781,True
2,HB103,South Waterfront Walkway - Sinatra Dr & 1 St,40.736982,-74.027781,True
3,JC014,Columbus Drive,40.718355,-74.038914,True
4,HB103,South Waterfront Walkway - Sinatra Dr & 1 St,40.736982,-74.027781,True
...,...,...,...,...,...
3681304,6608.09,E 54 St & 1 Ave,40.756265,-73.964179,False
3681305,5989.02,W 15 St & 6 Ave,40.738046,-73.996430,False
3681306,6438.04,11 St & 43 Ave,40.751907,-73.947912,False
3681307,5303.08,Canal St & Rutgers St,40.714275,-73.989900,False


In [4]:
idxd = station_entries.set_index('Station ID')
names = idxd['Station Name']
lats = idxd['Latitude']
lngs = idxd['Longitude']

In [26]:
stations_hist = station_entries.groupby(['Station ID', 'Station Name', 'Latitude', 'Longitude']).size().rename('count').reset_index()
stations_hist.to_parquet(f's3/stations/llname/{month}.parquet')
stations_hist

,Station ID,Station Name,Latitude,Longitude,count
0,2733.03,67 St & Erik Pl,40.633025,-74.017024,1
1,2733.03,67 St & Erik Pl,40.633290,-74.016589,1
2,2733.03,67 St & Erik Pl,40.633332,-74.016577,1
3,2733.03,67 St & Erik Pl,40.633334,-74.016566,1
4,2733.03,67 St & Erik Pl,40.633338,-74.016550,1
...,...,...,...,...,...
664983,JC109,Bergen Ave & Sip Ave,40.731448,-74.064507,1
664984,JC109,Bergen Ave & Sip Ave,40.731463,-74.064198,1
664985,JC109,Bergen Ave & Sip Ave,40.731476,-74.063674,1
664986,JC109,Bergen Ave & Sip Ave,40.731502,-74.063244,1


In [13]:
def row_sketch(a):
    restsum = sum(a[1:])
    total = a[0] + restsum
    num = len(a)
    return {
        'mode_count': a[0],
        'second': a[1] if num > 1 else nan,
        'restsum': restsum,
        'total': total,
        'counts': a,
        'first/second': a[0] / a[1] if num > 1 else nan,
        'mode_pct': a[0] / total,
        'num': num,
    }

def mode_sketch(df, groupby, thresh=0.5):
    idx_name = df.index.name
    if not idx_name:
        raise RuntimeError('Index needs a name')
    if isinstance(groupby, str):
        groupby = [groupby]
    row_groups = df.reset_index().groupby([idx_name] + groupby)
    row_hist = row_groups.size().rename('count').reset_index()
    counts = row_hist.groupby(idx_name)['count'].apply(lambda s: list(reversed(sorted(s.values))))
    row_sketches = counts.apply(row_sketch).apply(Series)
    below_thresh = row_sketches[row_sketches.mode_pct < thresh]
    if not below_thresh.empty:
        stderr.write(f'{len(below_thresh)} index entries with mode_pct < {thresh}:\n{below_thresh}\n')
    annotated = (
        row_hist
        .sort_values([idx_name, 'count'], ascending=False)
        .drop_duplicates(subset=idx_name)
        .set_index(idx_name)
    )
    annotated = sxs(annotated, row_sketches).drop(columns=['count']).sort_values('mode_pct')
    return annotated

In [14]:
annotated_station_names = mode_sketch(names.to_frame(), 'Station Name')
annotated_station_names

,Station Name,mode_count,second,restsum,total,counts,first/second,mode_pct,num
Station ID,,,,,,,,,
6708.04,Broadway\t& W 48 St,16538,511.0,511,17049,"[16538, 511]",32.363992,0.970028,2
6535.04,W 34 St &\tHudson Blvd E,11898,226.0,226,12124,"[11898, 226]",52.646018,0.981359,2
5382.07,Forsyth St\t& Grand St,18437,211.0,211,18648,"[18437, 211]",87.379147,0.988685,2
5329.08,Murray St\t& West St,3899,36.0,36,3935,"[3899, 36]",108.305556,0.990851,2
4781.05,Nassau St\t& Duffield St,2975,25.0,25,3000,"[2975, 25]",119.000000,0.991667,2
...,...,...,...,...,...,...,...,...,...
7409.08,W 84 St & Broadway,7321,NaN,0,7321,[7321],NaN,1.000000,1
7414.17,3 Ave & E 100 St,2460,NaN,0,2460,[2460],NaN,1.000000,1
7432.04,W 90 St & Amsterdam Ave,4956,NaN,0,4956,[4956],NaN,1.000000,1


In [15]:
annotated_stations = mode_sketch(sxs(lats, lngs), ['Latitude', 'Longitude',])
annotated_stations

,Latitude,Longitude,mode_count,second,restsum,total,counts,first/second,mode_pct,num
Station ID,,,,,,,,,,
8356.04,40.848793,-73.903178,1773,2.0,1014,2787,"[1773, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",886.500000,0.636168,1014
8156.03,40.836179,-73.907301,3556,3.0,1972,5528,"[3556, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1185.333333,0.643271,1954
8160.03,40.835334,-73.915400,1109,2.0,605,1714,"[1109, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, ...",554.500000,0.647025,600
8156.01,40.837213,-73.908345,1316,2.0,697,2013,"[1316, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",658.000000,0.653751,695
6134.13,40.741120,-73.921037,2,1.0,1,3,"[2, 1]",2.000000,0.666667,2
...,...,...,...,...,...,...,...,...,...,...
7271.01,40.779770,-73.923162,2862,1.0,136,2998,"[2862, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2862.000000,0.954636,137
5779.11,40.730311,-73.980472,16704,24.0,776,17480,"[16704, 24, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...",696.000000,0.955606,753
6599.01,40.757284,-73.953600,5468,1.0,252,5720,"[5468, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5468.000000,0.955944,253


In [16]:
stations = sxs(
    annotated_station_names['Station Name'],
    annotated_stations[['Latitude', 'Longitude']],
)
stations

,Station Name,Latitude,Longitude
Station ID,,,
6708.04,Broadway\t& W 48 St,40.760177,-73.984868
6535.04,W 34 St &\tHudson Blvd E,40.755167,-74.000599
5382.07,Forsyth St\t& Grand St,40.717798,-73.993161
5329.08,Murray St\t& West St,40.715030,-74.012460
4781.05,Nassau St\t& Duffield St,40.698477,-73.983840
...,...,...,...
7409.08,W 84 St & Broadway,40.786795,-73.977112
7414.17,3 Ave & E 100 St,40.787721,-73.947283
7432.04,W 90 St & Amsterdam Ave,40.790179,-73.972889


### Create unifed "dockings" with melted start/end

In [111]:
starts = d[['Start Station ID']].copy()
start_times = d['Start Time']
ends = d[['End Station ID']].copy()
end_times = d['Stop Time']

starts['Day'] = start_times.dt.day
starts['Hour'] = start_times.dt.hour
starts['Weekday'] = start_times.dt.weekday
ends['Day'] = end_times.dt.day
ends['Hour'] = end_times.dt.hour
ends['Weekday'] = end_times.dt.weekday

starts = starts.rename(columns={'Start Station ID': 'Station ID'})
starts['Start'] = True
ends = ends.rename(columns={'End Station ID': 'Station ID'})
ends['Start'] = False
dockings = pd.concat([starts, ends])
dockings

,Station ID,Day,Hour,Weekday,Start
0,5351.07,28,0,6,True
1,6164.09,31,16,2,True
2,7748.02,27,23,5,True
3,6164.09,26,19,4,True
4,5938.11,25,22,3,True
...,...,...,...,...,...
115226,HB502,3,19,2,False
115227,HB603,31,18,2,False
115228,HB603,24,14,2,False
115229,HB502,24,19,2,False


#### JC overall start/end counts

In [112]:
jcs = dockings[dockings['Station ID'].str.match('(?:JC|HB)', na=False)]
jcs.Start.value_counts()

False    114840
True     114839
Name: Start, dtype: int64

In [113]:
jc_starts = jcs[jcs.Start].groupby('Station ID').size().rename('Count')
jc_llcs = jc_starts.to_frame().merge(stations, left_index=True, right_index=True)
jc_llcs

,Count,Station Name,Latitude,Longitude
Station ID,,,,
HB101,3505,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305
HB102,4545,Hoboken Terminal - River St & Hudson Pl,40.736068,-74.029127
HB103,4567,South Waterfront Walkway - Sinatra Dr & 1 St,40.736982,-74.027781
HB105,2911,City Hall - Washington St & 1 St,40.737360,-74.030970
HB201,2437,12 St & Sinatra Dr N,40.750604,-74.024020
HB202,2317,14 St Ferry - 14 St & Shipyard Ln,40.752961,-74.024353
HB203,1355,Bloomfield St & 15 St,40.754530,-74.026580
HB301,1367,4 St & Grand St,40.742258,-74.035111
HB302,1040,6 St & Grand St,40.744398,-74.034501


#### Build date range (incl. weekday vs. weekends)

In [114]:
dates = pd.date_range('2022-08-01', '2022-08-31')
week_dts = dates[dates.weekday < 5]
wknd_dts = dates[dates.weekday >= 5]
len(week_dts), len(wknd_dts)

(23, 8)

### Station Counts helper

In [121]:
from typing import Collection

def station_counts(*keys, jc=None, weekdays=None, daily_avg=None):
    keys = list(keys)
    df = dockings
    dts = dates
    if weekdays is not None:
        if weekdays is True:
            df = df[df.Weekday < 5]
            dts = week_dts
        elif weekdays is False:
            df = df[df.Weekday >= 5]
            dts = wknd_dts
        elif isinstance(weekdays, Collection):
            weekdays = set(weekdays)
            df = df[df.Weekday.apply(lambda wd: wd in weekdays)]
            dts = dts[dts.weekday.apply(lambda wd: wd in weekdays)]
        else:
            raise ValueError(f'Unrecognized weekdays: {weekdays}')

    counts = df.groupby(['Station ID'] + keys).size().rename('Count')
    counts = counts.reset_index().merge(stations, left_on='Station ID', right_index=True).sort_values('Count')
    cols = ['Station Name'] + keys + ['Count']
    if daily_avg is None:
        if 'Day' in keys or 'Weekday' in keys:
            daily_avg = False
        else:
            daily_avg = True
    if daily_avg:
        if 'Day' in keys or 'Weekday' in keys:
            raise ValueError("Can only compute daily averages if daily grouping is not applied")
        counts['Avg'] = counts['Count'] / len(dts)
        cols += ['Avg']

    counts = counts.set_index('Station ID')[cols]

    if jc is True:
        counts = counts[counts.index.to_series().str.match('(?:JC|HB)')]
    elif js is False:
        counts = counts[~counts.index.to_series().str.match('(?:JC|HB)')]

    return counts

#### JC counts, by {day,station}

In [122]:
jcd = station_counts('Day', jc=True)
jcd

,Station Name,Day,Count
Station ID,,,
JC082,Manila & 1st,17,1
JC107,Grant Ave & MLK Dr,7,1
JC094,Glenwood Ave,6,1
JC107,Grant Ave & MLK Dr,20,1
JC095,Bergen Ave,1,1
...,...,...,...
JC005,Grove St PATH,3,409
HB102,Hoboken Terminal - River St & Hudson Pl,5,410
JC005,Grove St PATH,17,413


#### Biggest {station,day}s in JC

In [123]:
jcd[jcd.index.to_series().str.startswith('JC')].iloc[-50:]

,Station Name,Day,Count
Station ID,,,
JC098,Washington St,17,216
JC052,Liberty Light Rail,13,216
JC008,Newport Pkwy,21,217
JC066,Newport PATH,7,219
JC008,Newport Pkwy,7,221
JC052,Liberty Light Rail,14,222
JC008,Newport Pkwy,6,222
JC008,Newport Pkwy,28,223
JC066,Newport PATH,6,224


### Monthly sum + Daily avg (all JC+HB)

In [124]:
sc_all = station_counts(jc=True)
sc_all

,Station Name,Count,Avg
Station ID,,,
JC107,Grant Ave & MLK Dr,149,4.806452
JC108,Bergen Ave & Stegman St,327,10.548387
JC063,Jackson Square,411,13.258065
JC051,Union St,559,18.032258
JC065,Dey St,616,19.870968
JC094,Glenwood Ave,646,20.838710
JC018,5 Corners Library,718,23.161290
JC059,Heights Elevator,879,28.354839
JC084,Communipaw & Berry Lane,1031,33.258065


### Monthly sum + Daily avg, weekdays only (all JC+HB)

In [125]:
sc_wkdy = station_counts(jc=True, weekdays=True)
sc_wkdy

,Station Name,Count,Avg
Station ID,,,
JC107,Grant Ave & MLK Dr,112,4.869565
JC108,Bergen Ave & Stegman St,235,10.217391
JC063,Jackson Square,296,12.869565
JC051,Union St,391,17.000000
JC094,Glenwood Ave,479,20.826087
JC065,Dey St,486,21.130435
JC018,5 Corners Library,493,21.434783
JC059,Heights Elevator,619,26.913043
JC084,Communipaw & Berry Lane,776,33.739130


### Monthly sum + Daily avg, weekends only (all JC+HB)

In [126]:
sc_wknd = station_counts(jc=True, weekdays=False)
sc_wknd

,Station Name,Count,Avg
Station ID,,,
JC107,Grant Ave & MLK Dr,37,4.625
JC108,Bergen Ave & Stegman St,92,11.500
JC063,Jackson Square,115,14.375
JC065,Dey St,130,16.250
JC094,Glenwood Ave,167,20.875
JC051,Union St,168,21.000
JC018,5 Corners Library,225,28.125
JC095,Bergen Ave,253,31.625
JC084,Communipaw & Berry Lane,255,31.875


### Busiest 5 stations

In [127]:
top_stations = sc_all.iloc[-5:]['Station Name']
top_stations
# top_stations = pd.concat([sc_all.iloc[-5:], sc_wkdy.iloc[-5:], sc_wknd.iloc[-5:]])
# top_stations.index.drop_duplicates()

Station ID
JC008                                    Newport Pkwy
HB101        Hoboken Terminal - Hudson St & Hudson Pl
HB102         Hoboken Terminal - River St & Hudson Pl
HB103    South Waterfront Walkway - Sinatra Dr & 1 St
JC005                                   Grove St PATH
Name: Station Name, dtype: object

### Daily dockings (start+end), top 5 JC+HB stations

In [138]:
ts = jcd[jcd.index.isin(top_stations.index)].copy()
ts['Date'] = ts.Day.apply(lambda d: to_dt('2022-08-%02d' % d).date()).astype('datetime64')
ts = ts.sort_values('Date')
ts['Weekday'] = 'Weekend'
ts.loc[ts['Date'].dt.weekday < 5, 'Weekday'] = 'Weekday'
ts

,Station Name,Day,Count,Date,Weekday
Station ID,,,,,
HB103,South Waterfront Walkway - Sinatra Dr & 1 St,1,234,2022-08-01,Weekday
JC005,Grove St PATH,1,330,2022-08-01,Weekday
HB102,Hoboken Terminal - River St & Hudson Pl,1,243,2022-08-01,Weekday
HB101,Hoboken Terminal - Hudson St & Hudson Pl,1,192,2022-08-01,Weekday
JC008,Newport Pkwy,1,170,2022-08-01,Weekday
...,...,...,...,...,...
JC008,Newport Pkwy,31,161,2022-08-31,Weekday
HB102,Hoboken Terminal - River St & Hudson Pl,31,338,2022-08-31,Weekday
HB101,Hoboken Terminal - Hudson St & Hudson Pl,31,219,2022-08-31,Weekday


In [139]:
fig = px.line(ts, x='Date', y='Count', color='Station Name', labels={'Count': 'Daily ride starts+stops'})
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bgcolor='rgba(0,0,0,0)' ,
))
fig

### Daily dockings (start+end), Grove St PATH

In [140]:
title = 'Grove St PATH Citibike Station, daily starts+ends'
fig = px.bar(
    ts[ts['Station Name'] == 'Grove St PATH'], 
    x='Date', y='Count', 
    color='Weekday', 
    labels={'Count': 'Daily ride starts+stops', 'Weekday': ''},
)
fig.update_layout(
    title={
        'text': title,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
)
fig

In [134]:
fig = px.scatter_mapbox(
    jc_llcs,
    lat="Latitude", lon="Longitude",
    hover_name="Station Name",
    hover_data=["Station Name", "Count"],
    size='Count',
    #color_discrete_sequence=["fuchsia"], zoom=3, height=300
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()